# "NFL Score Prediction"
> "Predict how many points each team will score in a game"
- toc: false
- branch: master
- badges: true
- comments: true
- categories: [fastpages, jupyter]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

This is a work in progress, something fun to do for a sport I'm interested in. Go Browns!

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [2]:
df = pd.read_csv("C:/NFL Prediction/Data Raw/season18.csv",index_col=0)

In [3]:
df.head()

,GameID,date,home,away,homescore,awayscore
1,2018090600,2018-09-06,PHI,ATL,18,12
2,2018090900,2018-09-09,BAL,BUF,47,3
3,2018090907,2018-09-09,NYG,JAX,15,20
4,2018090906,2018-09-09,NO,TB,40,48
5,2018090905,2018-09-09,NE,HOU,27,20


### Work out who won each game and create a record for each game and team combination

In [4]:
df['homewin'] = np.where(df['homescore']> df['awayscore'],1,0)
df['awaywin'] = np.where(df['homescore']< df['awayscore'],1,0)

In [5]:
df.head()

,GameID,date,home,away,homescore,awayscore,homewin,awaywin
1,2018090600,2018-09-06,PHI,ATL,18,12,1,0
2,2018090900,2018-09-09,BAL,BUF,47,3,1,0
3,2018090907,2018-09-09,NYG,JAX,15,20,0,1
4,2018090906,2018-09-09,NO,TB,40,48,0,1
5,2018090905,2018-09-09,NE,HOU,27,20,1,0


In [6]:
home = df.drop(['awaywin'],axis=1)

In [7]:
home = home.rename(columns={'home':'team','homescore':'PF','awayscore':'PA','homewin':'win','away':'opponent'})

In [8]:
home.head()

,GameID,date,team,opponent,PF,PA,win
1,2018090600,2018-09-06,PHI,ATL,18,12,1
2,2018090900,2018-09-09,BAL,BUF,47,3,1
3,2018090907,2018-09-09,NYG,JAX,15,20,0
4,2018090906,2018-09-09,NO,TB,40,48,0
5,2018090905,2018-09-09,NE,HOU,27,20,1


In [9]:
home['hometeam'] = 1 

In [10]:
home.tail()

,GameID,date,team,opponent,PF,PA,win,hometeam
252,2018123000,2018-12-30,BAL,CLE,26,24,1,1
253,2018123014,2018-12-30,LA,SF,48,32,1,1
254,2018123009,2018-12-30,PIT,CIN,16,13,1,1
255,2018123015,2018-12-30,SEA,ARI,27,24,1,1
256,2018123011,2018-12-30,TEN,IND,17,33,0,1


In [11]:
away = df.drop(['homewin'],axis=1)
away = away[['GameID','date','away','home','awayscore','homescore','awaywin']]
away = away.rename(columns={'away':'team','awayscore':'PF','homescore':'PA','awaywin':'win','home':'opponent'})
away['hometeam'] = 0
away.head()

,GameID,date,team,opponent,PF,PA,win,hometeam
1,2018090600,2018-09-06,ATL,PHI,12,18,0,0
2,2018090900,2018-09-09,BUF,BAL,3,47,0,0
3,2018090907,2018-09-09,JAX,NYG,20,15,1,0
4,2018090906,2018-09-09,TB,NO,48,40,1,0
5,2018090905,2018-09-09,HOU,NE,20,27,0,0


In [12]:
scores = pd.concat([home,away])

In [13]:
scores.shape

(512, 8)

In [14]:
scores = scores.sort_values(['team','date'])
scores = scores.reset_index(drop=True)

In [15]:
scores.head()

,GameID,date,team,opponent,PF,PA,win,hometeam
0,2018090909,2018-09-09,ARI,WAS,6,24,0,1
1,2018091609,2018-09-16,ARI,LA,0,34,0,0
2,2018092311,2018-09-23,ARI,CHI,14,16,0,1
3,2018093008,2018-09-30,ARI,SEA,17,20,0,1
4,2018100710,2018-10-07,ARI,SF,28,18,1,0


### Create moving average score variables

In [16]:
ma1 = scores.groupby('team')['PF'].shift().rolling(1).mean().reset_index()
ma2 = scores.groupby('team')['PF'].shift().rolling(2).mean().reset_index()
ma3 = scores.groupby('team')['PF'].shift().rolling(3).mean().reset_index()

In [17]:
scores['score1MA'] = ma1['PF']
scores['score2MA'] = ma2['PF']
scores['score3MA'] = ma3['PF']

In [18]:
scores

,GameID,date,team,opponent,PF,PA,win,hometeam,score1MA,score2MA,score3MA
0,2018090909,2018-09-09,ARI,WAS,6,24,0,1,NaN,NaN,NaN
1,2018091609,2018-09-16,ARI,LA,0,34,0,0,6.0,NaN,NaN
2,2018092311,2018-09-23,ARI,CHI,14,16,0,1,0.0,3.0,NaN
3,2018093008,2018-09-30,ARI,SEA,17,20,0,1,14.0,7.0,6.666667
4,2018100710,2018-10-07,ARI,SF,28,18,1,0,17.0,15.5,10.333333
5,2018101405,2018-10-14,ARI,MIN,17,27,0,0,28.0,22.5,19.666667
6,2018101800,2018-10-18,ARI,DEN,10,45,0,1,17.0,22.5,20.666667
7,2018102809,2018-10-28,ARI,SF,18,15,1,1,10.0,13.5,18.333333
8,2018111105,2018-11-11,ARI,KC,14,26,0,0,18.0,14.0,15.000000
9,2018111807,2018-11-18,ARI,OAK,21,23,0,1,14.0,16.0,14.000000


In [19]:
scores_2 = pd.merge(scores,scores,how='left', left_on=['GameID','opponent'], right_on = ['GameID','team'], suffixes= ("","_opponent"))

In [20]:
scores_2.head()

,GameID,date,team,opponent,PF,PA,win,hometeam,score1MA,score2MA,...,date_opponent,team_opponent,opponent_opponent,PF_opponent,PA_opponent,win_opponent,hometeam_opponent,score1MA_opponent,score2MA_opponent,score3MA_opponent
0,2018090909,2018-09-09,ARI,WAS,6,24,0,1,NaN,NaN,...,2018-09-09,WAS,ARI,24,6,1,0,NaN,NaN,NaN
1,2018091609,2018-09-16,ARI,LA,0,34,0,0,6.0,NaN,...,2018-09-16,LA,ARI,34,0,1,1,33.0,NaN,NaN
2,2018092311,2018-09-23,ARI,CHI,14,16,0,1,0.0,3.0,...,2018-09-23,CHI,ARI,16,14,1,0,24.0,23.5,NaN
3,2018093008,2018-09-30,ARI,SEA,17,20,0,1,14.0,7.0,...,2018-09-30,SEA,ARI,20,17,1,0,24.0,20.5,21.666667
4,2018100710,2018-10-07,ARI,SF,28,18,1,0,17.0,15.5,...,2018-10-07,SF,ARI,18,28,0,1,27.0,27.0,28.000000


In [21]:
scores_2 = scores_2.drop(['date_opponent','team_opponent','opponent_opponent','PF_opponent','PA_opponent','win_opponent','hometeam_opponent'],axis=1)

### Create win record coming into the game

In [86]:
scores_2['cum_wins'] = scores.groupby('team')['win'].transform(lambda x: x.cumsum().shift())
scores_2['cum_PF'] = scores.groupby('team')['PF'].transform(lambda x: x.cumsum().shift())
scores_2['cum_PA'] = scores.groupby('team')['PA'].transform(lambda x: x.cumsum().shift())
scores_2['cum_games'] = scores.groupby('team').cumcount()
scores_2['cum_wins'] = scores_2['cum_wins'].fillna(0)
scores_2['win_pct'] = scores_2['cum_wins']/scores_2['cum_games']

In [87]:
scores_2

,GameID,date,team,opponent,PF,PA,win,hometeam,score1MA,score2MA,score3MA,score1MA_opponent,score2MA_opponent,score3MA_opponent,cum_wins,cum_games,win_pct,cum_PF,cum_PA
0,2018090909,2018-09-09,ARI,WAS,6,24,0,1,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0,NaN,NaN,NaN
1,2018091609,2018-09-16,ARI,LA,0,34,0,0,6.0,0.0,0.000000,33.0,0.0,0.000000,0.0,1,0.000000,6.0,24.0
2,2018092311,2018-09-23,ARI,CHI,14,16,0,1,0.0,3.0,0.000000,24.0,23.5,0.000000,0.0,2,0.000000,6.0,58.0
3,2018093008,2018-09-30,ARI,SEA,17,20,0,1,14.0,7.0,6.666667,24.0,20.5,21.666667,0.0,3,0.000000,20.0,74.0
4,2018100710,2018-10-07,ARI,SF,28,18,1,0,17.0,15.5,10.333333,27.0,27.0,28.000000,0.0,4,0.000000,37.0,94.0
5,2018101405,2018-10-14,ARI,MIN,17,27,0,0,28.0,22.5,19.666667,23.0,27.0,20.000000,1.0,5,0.200000,65.0,112.0
6,2018101800,2018-10-18,ARI,DEN,10,45,0,1,17.0,22.5,20.666667,20.0,18.0,19.666667,1.0,6,0.166667,82.0,139.0
7,2018102809,2018-10-28,ARI,SF,18,15,1,1,10.0,13.5,18.333333,10.0,20.0,19.333333,1.0,7,0.142857,92.0,184.0
8,2018111105,2018-11-11,ARI,KC,14,26,0,0,18.0,14.0,15.000000,37.0,33.5,37.333333,2.0,8,0.250000,110.0,199.0
9,2018111807,2018-11-18,ARI,OAK,21,23,0,1,14.0,16.0,14.000000,6.0,4.5,12.333333,2.0,9,0.222222,124.0,225.0


In [104]:
win_percentage = scores_2.filter(['GameID','date','team','win_pct','cum_wins','cum_PF','cum_PA'])

In [105]:
scores_3 = pd.merge(scores_2,win_percentage,how='left',left_on=['date','opponent'],right_on=['date','team'],suffixes=('','_opponent'))

In [106]:
scores_3 = scores_3.drop(['GameID_opponent','team_opponent'],axis=1)
scores_3['win_pct_var'] = scores_3['win_pct'] - scores_3['win_pct_opponent']
scores_3['cum_wins_var'] = scores_3['cum_wins'] - scores_3['cum_wins_opponent']
scores_3['cum_PF_cum_PA_opp'] = scores_3['cum_PF'] - scores_3['cum_PA_opponent']
scores_3['cum_PA_cum_PF_opp'] = scores_3['cum_PA'] - scores_3['cum_PF_opponent']
scores_3.head()

,GameID,date,team,opponent,PF,PA,win,hometeam,score1MA,score2MA,...,cum_PF,cum_PA,win_pct_opponent,cum_wins_opponent,cum_PF_opponent,cum_PA_opponent,win_pct_var,cum_wins_var,cum_PF_cum_PA_opp,cum_PA_cum_PF_opp
0,2018090909,2018-09-09,ARI,WAS,6,24,0,1,0.0,0.0,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN
1,2018091609,2018-09-16,ARI,LA,0,34,0,0,6.0,0.0,...,6.0,24.0,1.000000,1.0,33.0,13.0,-1.000000,-1.0,-7.0,-9.0
2,2018092311,2018-09-23,ARI,CHI,14,16,0,1,0.0,3.0,...,6.0,58.0,0.500000,1.0,47.0,41.0,-0.500000,-1.0,-35.0,11.0
3,2018093008,2018-09-30,ARI,SEA,17,20,0,1,14.0,7.0,...,20.0,74.0,0.333333,1.0,65.0,64.0,-0.333333,-1.0,-44.0,9.0
4,2018100710,2018-10-07,ARI,SF,28,18,1,0,17.0,15.5,...,37.0,94.0,0.250000,1.0,100.0,118.0,-0.250000,-1.0,-81.0,-6.0


#### Import player stats for each game

In [24]:
player = pd.read_csv("C:/NFL Prediction/Data Raw/ps18.csv",index_col=0)

In [25]:
player.head()

,Season,game.id,date,Team,playerID,name,pass.att,pass.comp,passyds,pass.tds,...,asst.tackles,sacks,defints,forced.fumbs,totalfumbs,recfumbs,totalrecfumbs,fumbyds,fumbslost,games
1,2018,2018090600,2018-09-06,PHI,00-0029567,N.Foles,34,19,117,0,...,0,0.0,0,0,1,1,1,0,0,1
2,2018,2018090600,2018-09-06,PHI,00-0033731,T.Sullivan,0,0,0,0,...,0,0.0,0,0,1,0,0,0,1,1
3,2018,2018090600,2018-09-06,PHI,00-0023564,D.Sproles,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,1
4,2018,2018090600,2018-09-06,PHI,00-0033573,S.Gibson,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,1
5,2018,2018090600,2018-09-06,PHI,00-0031549,N.Agholor,1,1,15,0,...,0,0.0,0,0,0,0,0,0,0,1


In [26]:
player.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13649 entries, 1 to 13649
Data columns (total 57 columns):
Season           13649 non-null int64
game.id          13649 non-null int64
date             13649 non-null object
Team             13624 non-null object
playerID         13649 non-null object
name             13649 non-null object
pass.att         13649 non-null int64
pass.comp        13649 non-null int64
passyds          13649 non-null int64
pass.tds         13649 non-null int64
pass.ints        13649 non-null int64
pass.twopta      13649 non-null int64
pass.twoptm      13649 non-null int64
rush.att         13649 non-null int64
rushyds          13649 non-null int64
rushtds          13649 non-null int64
rushlng          13649 non-null int64
rushlngtd        13649 non-null int64
rush.twopta      13649 non-null int64
rush.twoptm      13649 non-null int64
recept           13649 non-null int64
recyds           13649 non-null int64
rec.tds          13649 non-null int64
reclng       

#### Aggregate the columns to team and date level

In [27]:
team = player.groupby(['Team','date']).sum().reset_index()

In [28]:
team.head()

,Team,date,Season,game.id,pass.att,pass.comp,passyds,pass.tds,pass.ints,pass.twopta,...,asst.tackles,sacks,defints,forced.fumbs,totalfumbs,recfumbs,totalrecfumbs,fumbyds,fumbslost,games
0,ARI,2018-09-09,46414,46416090907,34,20,153,0,1,1,...,15,3.0,0,2,1,0,1,0,1,23
1,ARI,2018-09-16,46414,46416107007,27,17,90,0,1,0,...,19,2.0,1,0,0,0,0,0,0,23
2,ARI,2018-09-23,50450,50452307775,26,17,193,2,3,0,...,16,3.0,1,1,2,1,2,0,1,25
3,ARI,2018-09-30,54486,54488511216,27,15,180,1,0,0,...,1,2.0,0,2,2,1,1,0,1,27
4,ARI,2018-10-07,46414,46416316330,25,10,170,1,0,0,...,21,4.0,2,3,1,1,4,72,0,23


# Run Model

In [107]:
scores_3.fillna(0, inplace=True)

X=scores_3[['score1MA','hometeam','cum_wins_var','cum_PF','cum_PA','cum_PF_cum_PA_opp','cum_PA_cum_PF_opp']]  # Features
y=scores_3['win']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [110]:
clf=RandomForestClassifier(n_estimators=100,min_samples_leaf=20)
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

0.5454545454545454

In [111]:
clf.score(X_train,y_train)

0.723463687150838

In [112]:
feature_importances = pd.DataFrame(clf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

,importance
cum_PF_cum_PA_opp,0.196144
score1MA,0.163509
cum_PA_cum_PF_opp,0.152062
cum_PA,0.146758
hometeam,0.136369
cum_PF,0.118839
cum_wins_var,0.086319


In [33]:
team.corr()

,Season,game.id,pass.att,pass.comp,passyds,pass.tds,pass.ints,pass.twopta,pass.twoptm,rush.att,...,asst.tackles,sacks,defints,forced.fumbs,totalfumbs,recfumbs,totalrecfumbs,fumbyds,fumbslost,games
Season,1.000000,1.000000,0.094738,0.059402,0.066156,0.022737,0.029400,-0.000546,-0.067025,-0.056462,...,0.156220,0.007156,0.024758,-0.061828,0.037763,0.070571,0.023604,-0.030357,-0.003442,1.000000
game.id,1.000000,1.000000,0.094729,0.059393,0.066143,0.022729,0.029396,-0.000548,-0.067027,-0.056459,...,0.156224,0.007155,0.024755,-0.061833,0.037759,0.070567,0.023600,-0.030355,-0.003443,1.000000
pass.att,0.094738,0.094729,1.000000,0.871394,0.662993,0.092164,0.282843,0.272143,0.168898,-0.438757,...,0.019647,-0.140815,-0.157295,-0.081766,0.114542,-0.014914,-0.081599,-0.139443,0.223178,0.094738
pass.comp,0.059402,0.059393,0.871394,1.000000,0.776024,0.243571,0.122287,0.260056,0.161179,-0.365487,...,-0.042829,-0.120367,-0.168576,-0.080259,0.136556,-0.003809,-0.089594,-0.126877,0.228164,0.059402
passyds,0.066156,0.066143,0.662993,0.776024,1.000000,0.523539,0.093346,0.254015,0.200639,-0.225678,...,-0.017563,-0.044256,-0.123188,-0.052112,0.064924,-0.017414,-0.056500,-0.103763,0.118211,0.066156
pass.tds,0.022737,0.022729,0.092164,0.243571,0.523539,1.000000,-0.095620,0.140548,0.155166,0.050371,...,-0.001678,0.082012,0.078079,0.062989,-0.046482,-0.031253,0.032131,-0.017377,-0.041184,0.022737
pass.ints,0.029400,0.029396,0.282843,0.122287,0.093346,-0.095620,1.000000,0.058614,0.040833,-0.181914,...,0.098483,-0.082520,-0.083441,-0.025834,-0.004360,-0.017912,-0.027767,-0.069647,0.032185,0.029400
pass.twopta,-0.000546,-0.000548,0.272143,0.260056,0.254015,0.140548,0.058614,1.000000,0.673605,-0.146960,...,-0.065502,-0.049449,-0.084848,-0.058847,0.031043,-0.021044,-0.072585,-0.000543,0.084078,-0.000546
pass.twoptm,-0.067025,-0.067027,0.168898,0.161179,0.200639,0.155166,0.040833,0.673605,1.000000,-0.077365,...,-0.038787,-0.081839,-0.071604,-0.036035,-0.024922,-0.038424,-0.067395,-0.017553,0.014507,-0.067025
rush.att,-0.056462,-0.056459,-0.438757,-0.365487,-0.225678,0.050371,-0.181914,-0.146960,-0.077365,1.000000,...,-0.239365,0.162031,0.229744,0.073483,0.019182,0.106455,0.172945,0.018136,-0.113000,-0.056462
